<img src="http://www.cidaen.es/assets/img/mCIDaeNnb.png" alt="Logo CiDAEN" align="right">

<h1><font size=4>Trabajo Fin de Master (TFM)</font></h1>
<br>
<h2><font size=6>WiDS Datathon 2024 - Challenge 2</font></h2>
<h3><font size=5>Modelos de regresión para estimación del periodo de diagnóstico metastático</font></h3>
<h3><font size=5>Parte 2 - Modelos de Regresión</font></h3>
<br>
<h1><font size=4>Alumna: Luna Jiménez Fernández</font></h1>
<br>



<div align="right">
<font size=3>Máster en Ciencia de Datos e Ingeniería de Datos en la Nube</font><br>
<font size=3>Universidad de Castilla-La Mancha</font>
</div>

<br>

---

In [182]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:98% !important; }</style>"))

# Array manipulation libraries
import numpy as np
import pandas as pd

# Pre-processing
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.preprocessing import RobustScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

# Regression models
from sklearn.linear_model import Lasso
from sklearn.ensemble import RandomForestRegressor

# Importing visualization libraries
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

%config InlineBackend.figure_format = 'retina'
%matplotlib inline

In [183]:
# Seed for random experiments - 7 is the number
RANDOM_SEED = 777

En la primera libreta se realizó un **análisis exploratorio de datos** exhaustivo para entender en profundidad el comportamiento del conjunto de datos de interés - el **segundo desafío** del **Women in Data Science (*WiDS*) Datathon** del año 2024, disponible en el [siguiente enlace](https://www.kaggle.com/competitions/widsdatathon2024-challenge2/overview).

Tras este estudio, el objetivo de la siguiente libreta es tanto la **construcción de modelos de regresión** capaces de predecir el **tiempo de diagnóstico de la metástasis** a partir de los atributos seleccionados, como la **evaluación** de estos con el fin de estudiar si resultan de utilidad y si - como se planteó - los **atributos geográficos, socioeconómicos y climáticos** juegan algún papel relevante en la estimación de los valores.

---

# Índice

* [3. Selección de atributos y pre-procesamiento](#section3)
    * [3.1. Carga y particionamiento del conjunto de datos](#section3-1)
    * [3.2. Selección de atributos](#section3-2)
    * [3.3. Pre-procesamiento de los atributos seleccionados](#section3-3)
* [4. Selección de modelos de regresión e hiperparámetros](#section4)
* [5. Experimentación](#section5)
* [6. Análisis de resultados](#section6)
* [7. Conclusiones](#section7)
---

<a id="section3"></a>

# 3. Selección de atributos y pre-procesamiento

Tras finalizar el **análisis exploratorio de datos** en la libreta anterior, el siguiente paso en el proceso de ciencia de datos es el **preprocesamiento de la información** - para ser utilizada posteriormente por modelos de regresión, con el fin de predecir el tiempo de diagnóstico de la metástasis.

Concretamente, en este apartado se realizan las siguientes preparaciones:
- **Cargar y particionar** los conjuntos de datos en **entrenamiento**, **validación** y **test**.
- **Seleccionar el subconjunto de atributos** que van a ser utilizados durante la experimentación.
- **Preparar las *pipelines*** encargadas de transformar los datos crudos en datos listos para ser utilizados por los modelos posteriores.

---

<a id="section3-1"></a>

## 3.1. Carga y particionamiento del conjunto de datos

Durante el análisis exploratorio de datos se trabajó únicamente sobre el **conjunto de entrenamiento** - con el fin de evitar cualquier posible fuga de datos al estudiar el conjunto de test. Ahora bien, el desafio en Kaggle ofrece **dos conjuntos de datos**:
- `train.csv`: El **conjunto de entrenamiento**, con **150 atributos** y los valores de la **variable objetivo** (el tiempo de diagnóstico) asociados a cada instancia.
- `test.csv`: El **conjunto de test**, conteniendo únicamente los **150 atributos** sin los valores de la variable objetivo.

El primer paso, por tanto, consiste en **cargar ambos conjuntos de datos** como *DataFrames*:

In [184]:
# Loading the CSV files - force zip3 to be read as a string
df_train = pd.read_csv("data/train.csv", index_col="patient_id", dtype={"patient_zip3": object})
df_test = pd.read_csv("data/test.csv", index_col="patient_id", dtype={"patient_zip3": object})

# Display a small sample of both datasets to show that they have been properly loaded
print(f"Training size: {df_train.shape}")
display(df_train.sample(5))
print(f"Test size: {df_test.shape}")
display(df_test.sample(5))

Training size: (13173, 151)


,patient_race,payer_type,patient_state,patient_zip3,Region,Division,patient_age,patient_gender,bmi,breast_cancer_diagnosis_code,...,Average of Apr-18,Average of May-18,Average of Jun-18,Average of Jul-18,Average of Aug-18,Average of Sep-18,Average of Oct-18,Average of Nov-18,Average of Dec-18,metastatic_diagnosis_period
patient_id,,,,,,,,,,,,,,,,,,,,,
554288,White,MEDICAID,CA,926,West,Pacific,42,F,NaN,1749,...,63.97,64.32,67.87,77.78,77.55,73.11,69.83,64.83,58.61,227
275724,White,MEDICARE ADVANTAGE,TX,782,South,West South Central,71,F,NaN,C50111,...,66.76,86.57,85.59,85.34,85.54,79.74,68.93,56.25,53.16,70
857386,NaN,MEDICARE ADVANTAGE,IN,471,Midwest,East North Central,78,F,29.02,C50912,...,50.42,73.79,77.17,77.84,77.60,73.75,59.60,39.97,39.11,28
859329,NaN,COMMERCIAL,NY,109,Northeast,Middle Atlantic,41,F,NaN,C50112,...,43.04,62.25,66.80,73.77,73.91,66.31,51.92,38.02,33.12,244
564373,NaN,COMMERCIAL,TX,770,South,West South Central,43,F,NaN,C50112,...,66.50,79.54,84.57,84.85,85.09,80.61,72.46,58.14,55.77,135


Test size: (5646, 150)


,patient_race,payer_type,patient_state,patient_zip3,Region,Division,patient_age,patient_gender,bmi,breast_cancer_diagnosis_code,...,Average of Mar-18,Average of Apr-18,Average of May-18,Average of Jun-18,Average of Jul-18,Average of Aug-18,Average of Sep-18,Average of Oct-18,Average of Nov-18,Average of Dec-18
patient_id,,,,,,,,,,,,,,,,,,,,,
930449,White,MEDICAID,NY,112,Northeast,Middle Atlantic,47,F,22.00,1748,...,37.96,47.69,65.17,70.35,77.20,77.82,69.92,55.95,42.18,37.27
325814,NaN,MEDICARE ADVANTAGE,MN,566,Midwest,West North Central,73,F,NaN,C50912,...,25.99,31.19,58.79,65.30,68.94,66.94,56.30,38.58,20.20,18.29
765950,Asian,MEDICAID,MN,551,Midwest,West North Central,32,F,33.20,C50411,...,31.07,34.24,65.86,71.39,73.60,72.33,64.00,44.23,27.80,25.81
863403,Asian,MEDICARE ADVANTAGE,KY,406,South,East South Central,76,F,31.64,1749,...,43.13,50.08,72.30,76.32,77.86,76.91,74.87,59.80,43.30,40.62
351421,NaN,COMMERCIAL,VA,230,South,South Atlantic,42,F,NaN,C50412,...,42.64,54.94,71.89,75.26,76.76,77.90,75.38,61.27,46.32,41.33


Ahora bien, debido al proceso que se va a seguir durante el entrenamiento de los modelos (**selección de hiperparámetros**, **selección de modelos** y **evaluación**), utilizar directamente los conjuntos de datos descritos podría llevar a un problema de **fuga de datos** - al usar el mismo conjunto de datos para entrenar los modelos y evaluar sus hiperparámetros.

Para evitar esto, se va a dividir el conjunto de entrenamiento en dos - un conjunto de **entrenamiento** y uno de **validación**  -, siendo la distribución final la siguiente:
- **Entrenamiento**: El conjunto de entrenamiento cumple dos tareas - tanto el **entrenamiento de los modelos de regresión** propuestos como el **ajuste de hiperparámetros de los mismos** a través de una validación cruzada.
- **Validación**: Una vez se tienen los modelos entrenados, el conjunto de validación será utilizado para **seleccionar el mejor modelo de forma honesta** - utilizando un conjunto de datos que no han utilizado durante el entrenamiento para evitar sesgos o fugas de datos.
- **Test**: Finalmente, se utilizará el conjunto de test para **evaluar el rendimiento real** del modelo seleccionado a través de la validación - utilizando una plataforma externa (***Kaggle***) para medir este rendimiento. 

Además, todos estos conjuntos de datos se van a fraccionar en **atributos** (`X`) y **variable objetivo** (`y`) para seguir el estándar de `scikit-learn`.

In [185]:
# Split the datasets into training, validation and test
# Train / Val
X, y = df_train.drop(columns="metastatic_diagnosis_period"), df_train["metastatic_diagnosis_period"]
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.25, random_state=RANDOM_SEED)

# Test
X_test = df_test

# Display the information about each dataset - to ensure that it has been loaded and partitioned correctly
print("ENTRENAMIENTO:")
print(f"\t-Atributos: {X_train.shape}")
print(f"\t-Variable objetivo: {y_train.shape}")
print("VALIDACIÓN:")
print(f"\t-Atributos: {X_val.shape}")
print(f"\t-Variable objetivo: {y_val.shape}")
print("TEST:")
print(f"\t-Atributos: {X_test.shape}")

ENTRENAMIENTO:
	-Atributos: (9879, 150)
	-Variable objetivo: (9879,)
VALIDACIÓN:
	-Atributos: (3294, 150)
	-Variable objetivo: (3294,)
TEST:
	-Atributos: (5646, 150)


---

<a id="section3-2"></a>

## 3.2. Selección de atributos

Como se observó durante el análisis exploratorio, no tendría sentido utilizar directamente el **conjunto de datos completos** para el entrenamiento de modelos:
- La **dimensionalidad del conjunto de datos** - con 150 atributos en total - es excesiva para la cantidad de datos disponible, lo que podría llevar a sobreajustes.
- Algunos atributos tienen **una cantidad excesiva de posibles valores** - que se puede traducir, de nuevo, en sobreajustes del modelo al no tener suficientes datos para aprender adecuadamente las relaciones.
- La **amplia mayoría de atributos son irrelevantes** para la variable objetivo - ya sea por su baja calidad o por la poca correlación que tienen con la variable objetivo.

Por tanto, es necesario realizar una **selección de un subconjunto de atributos** para reducir la dimensionalidad y cribar los atributos que no sean relevantes para la predicción. Para buscar este subconjunto, se proponen varias opciones:

### 3.2.1. Selección por análisis exploratorio

Durante el análisis exploratorio de datos se ha realizado un análisis exhaustivo de los datos - tanto su **comportamiento** como su **relevancia** y las **transformaciones** que serían necesarias para utilizarse.

A partir de las conclusiones extraidas, se ha obtenido el siguiente **conjunto de atributos** - representando los atributos más relevantes estudiados dentro del conjunto de datos, junto a las **transformaciones a aplicar** sobre éstos:
- **Código de diagnóstico del cancer de mama (`breast_cancer_diagnosis_code`):** Variable categórica.
    - Debido al gran número de posibles valores, es necesario **agrupar los valores menos frecuentes**.
- **Código de diagnóstico del cancer metastático (`metastatic_cancer_diagnosis_code`):** Variable categórica.
    - Debido al gran número de posibles valores, es necesario **agrupar los valores menos frecuentes**.
- **Estado de residencia del paciente (`patient_state`):** Variable categórica.
    - Debido al gran número de posibles valores, es necesario **agrupar los valores menos frecuentes**.
- **Raza del paciente (`patient_race`):** Variable categórica.
    - Se **agrupan los valores perdidos** bajo un único valor.
- **Tipo de seguro médico del paciente (`payer_type`):** Variable categórica.
    - Se **agrupan los valores perdidos** bajo un único valor.

In [186]:
categorical_manual = [
    "breast_cancer_diagnosis_code",
    "metastatic_cancer_diagnosis_code",
    "patient_race",
    "payer_type",
    "patient_state"
]

Estas transformaciones se han elegido en base a los **test estadísticos** que se realizaron durante el análisis exploratorio:
- La **agrupación de los valores** en las variables de alta dimensionalidad aumenta la significación estadística, al reducirse el número de valores con un número demasiado bajo de instancias.
- La **sustitución de valores perdidos** mejora el rendimiento en los atributos donde el número de valores perdidos es excesivo.

A su vez, se ha optado por descartar los siguientes atributos:
- **Edad (`patient_age`) y IMC (`bmi`) del paciente**: Variables numéricas sin correlación con la variable objetivo.
- **Región (`Region`) y división (`Division`) del paciente**: Variables categóricas con poca relevancia, y ya representadas por otra variable más significativa (`patient_state`).
- **Código zip del paciente (`patient_zip3`)**: Variable categórica ya representada por otra variable (`patient_state`) con dimensionalidad excesiva.
- **Todas las variables geográficas, socioeconómicas y climáticas**: 136 atributos numéricos sin correlación con la variable objetivo.

### 3.2.2. Selección automática

En el apartado anterior se ha obtenido un subconjunto de atributos en base al estudio que se realizó en la libreta anterior. Ahora bien, aunque el estudio estuviese apoyado en **gráficas y tests estadísticos**, sigue existiendo la posibilidad de que **el analista haya introducido sesgos propios**.

Otra posibilidad es utilizar **algoritmos de aprendizaje automático** para realizar el proceso de selección de atributos - ya sea basandose en **tests estadísticos** o en el **comportamiento de modelos reales entrenados sobre los datos**.

Ahora bien, la mayoría de métodos necesitan un **preprocesamiento previo**, para imputar valores perdidos y unificar el comportamiento de atributos categóricos y numéricos. Concretamente, en este caso:
- **Atributos categóricos**: Se imputan los valores perdidos como un valor constante (`UNKNOWN`) y se **codifican los posibles valores del atributo** utilizando `One-Hot Encoding` - donde cada valor de cada variable categórica se **representa como un atributo separado**, que puede ser `True` (si el valor de la variable se corresponde) o `False` en cualquier otro caso.
    - Debido a la gran complejidad de algunos atributos, se opta por **agrupar todos los valores con menos de 100 instancias**. Este valor se ha elegido de forma arbitraria para la selección de variables, y tendrá que ser ajustado posteriormente durante el ajuste de hiperparámetros.
    - La codificación va a causar que **aumente el número de atributos respecto al conjunto de entrenamiento original** - lo que significa que **será necesario re-agrupar los valores de las variables mediante estadísticos** para realizar la selección.
- **Atributos numéricos**: Se imputan los valores perdidos utilizando la **mediana** (debido al gran número de valores extremos) y se **escalan los valores** utilizando la mediana y la desviación estándar.

Se define un `Pipeline` para realizar automáticamente el preprocesamiento descrito - aplicandose sobre los datos del **conjunto de entrenamiento** para evitar fugas de datos:

In [187]:
# Column types
categorical_variables = X_train.select_dtypes(exclude=np.number).columns.to_list()
numerical_variables = X_train.select_dtypes(include=np.number).columns.to_list()

# Feature Selection pipeline - Transforms the dataset to allow for feature selection
# NOTE: No categorical variable grouping is performed - which may lead to a massive number of attributes

# Categorical transformations - imputing and one-hot encoding
fs_categorical = Pipeline([
    ("imputer", SimpleImputer(strategy="constant", fill_value="UNKNOWN")),
    ("ohencoder", OneHotEncoder(min_frequency=100))
])

# Numerical transformations - imputing and scaling
fs_numerical = Pipeline([
    ("imputer", SimpleImputer(strategy="median")),
    ("scaler", RobustScaler())
])

# Final pre-processing
fs_preprocessing = ColumnTransformer([
    ("cat", fs_categorical, categorical_variables),
    ("num", fs_numerical, numerical_variables)
])

Existen dos estrategias para realizar una selección automática de atributos:

#### - Métodos *filter*

Los **métodos de filtrado (*filter*)** utilizan **test estadísticos** para evaluar de forma automática la relevancia de cada variable, sin la necesidad de entrenar modelos- haciendolos **más agiles**, aunque menos capaces de identificar las correlaciones entre grupos de atributos.

Para este caso se utilizan **tests F** - tests de **varianza**, para identificar las **diez variables** más relevantes del conjunto de datos 

In [188]:
# FEATURE RANKING - Using F-Regression
# Create the pipeline
fs_kbest = Pipeline([
    ("preprocessing", fs_preprocessing),
    ("kbest", SelectKBest(f_regression, k="all"))
])
fs_kbest.fit(X_train, y_train)

# Join the variable names and values
df_kbest = pd.DataFrame({
    "variable": fs_kbest.get_feature_names_out(),
    "score": fs_kbest["kbest"].scores_
})

# Process the dataframe, step by step
df_kbest_ordered = (
    df_kbest.assign(variable=(
        df_kbest["variable"].str.extract(r"cat__(?P<cat>[0-9A-Za-z_\- ]+)_(?:[0-9A-Za-z\- ]+|infrequent)|num__(?P<num>[0-9A-Za-z_\- ]+)")   # 1 - Extract the proper variable name
        .apply(lambda s: s["cat"] if not pd.isna(s["cat"]) else s["num"], axis=1)                                                           #     (joining into a single column)
    ))
    .groupby("variable").agg(max_score=("score", "max"), avg_score=("score", "mean"))                                                       # 2 - Obtain the maximum and avg score of each variable
    .sort_values(by="max_score", ascending=False)                                                                                           # 3 - Sort by MAXIMUM score
    
)

# Display the 15 best attributes
display(df_kbest_ordered.head(15))

,max_score,avg_score
variable,,
breast_cancer_diagnosis_desc,3163.104702,293.539683
breast_cancer_diagnosis_code,3163.104702,293.539683
metastatic_cancer_diagnosis_code,58.998175,12.662202
payer_type,56.152011,21.747065
patient_age,32.205066,32.205066
breast_cancer_diagnosis_desc_infrequent,30.799057,30.799057
breast_cancer_diagnosis_code_infrequent,30.799057,30.799057
patient_state,28.378242,3.208365
labor_force_participation,15.166246,15.166246


A partir de estos atributos obtenidos, se observa que:
- Los **atributos más importantes** son consistentes con los observados durante el estudio: **el código de diagnóstico de cancer** - tanto original como metástasis - y el **tipo de seguro médico**.
- El algoritmo da **mayor importancia** a algunos atributos numéricos - principalmente:
    - **Edad del paciente** (`patient_age`)
    - **Porcentaje de residentes empleados** (`labor_force_participation`)
    - **Porcentaje de residentes con estudios universitarios** (`education_bachelors` y `education_college_or_above`)
    - **Porcentaje de hogares con dos o más ingresos** (`family_dual_income`)
- Algunos atributos categóricos estudiados **tienen menor importancia de la esperada**:
    - **Estado de residencia del paciente** (`patient_state`)
    - **Raza del paciente** (`patient_race`)

Es de interés destacar que **existe una diferencia muy considerable en la importancia de los atributos** - donde el **tipo de cancer de mama original** es varios ordenes de magnitud más relevante que el resto de variable.

El **subconjunto de variables** obtenido por el proceso es el siguiente:

In [189]:
categorical_filter = ["breast_cancer_diagnosis_code", "metastatic_cancer_diagnosis_code", "payer_type", "patient_state", "patient_race"]
numerical_filter = ["patient_age", "labor_force_participation", "education_bachelors", "family_dual_income", "education_college_or_above"]

#### - Métodos *wrapper*

Los **métodos de envoltura *wrapper*** utilizan **modelos de aprendizaje automáticos** entrenados sobre los datos para elegir las variables más relevantes. Pese a ser más lentos, sus resultados son **más fiables** - al estar estudiando el comportamiento real de un modelo.

Para este caso se entrenará un modelo de **Random Forest**, utilizando el **conjunto de entrenamiento** para evitar fugas - buscando encontrar los **diez atributos más relevantes**:

In [190]:
# FEATURE RANKING - Using Random Forest as a wrapper
# Create the pipeline
fs_rf = Pipeline([
    ("preprocessing", fs_preprocessing),
    ("wrapper", RandomForestRegressor(n_estimators=100, random_state=RANDOM_SEED))
])
fs_rf.fit(X_train, y_train)

# Join the variable names and values
df_rf = pd.DataFrame({
    "variable": fs_rf["preprocessing"].get_feature_names_out(),
    "score": fs_rf["wrapper"].feature_importances_
})

# Process the dataframe, step by step
df_rf_ordered = (
    df_rf.assign(variable=(
        df_rf["variable"].str.extract(r"cat__(?P<cat>[0-9A-Za-z_\- ]+)_(?:[0-9A-Za-z\- ]+|infrequent)|num__(?P<num>[0-9A-Za-z_\- ]+)")      # 1 - Extract the proper variable name
        .apply(lambda s: s["cat"] if not pd.isna(s["cat"]) else s["num"], axis=1)                                                           #     (joining into a single column)
    ))
    .groupby("variable").agg(max_score=("score", "max"), avg_score=("score", "mean"))                                                       # 2 - Obtain the maximum and avg score of each variable
    .sort_values(by="max_score", ascending=False)                                                                                           # 3 - Sort by MAXIMUM score
    
)

# Display the 15 best attributes
display(df_rf_ordered.head(15))

,max_score,avg_score
variable,,
breast_cancer_diagnosis_code,0.123410,0.012368
breast_cancer_diagnosis_desc,0.119221,0.012102
patient_age,0.079427,0.079427
bmi,0.037954,0.037954
metastatic_cancer_diagnosis_code,0.012096,0.004549
breast_cancer_diagnosis_desc_infrequent,0.010563,0.010563
breast_cancer_diagnosis_code_infrequent,0.009509,0.009509
payer_type,0.007699,0.006449
patient_race,0.007403,0.005098


*(NOTA: `RandomForest` tiene un componente aleatorio. Se ha utilizado una semilla, pero sigue existiendo la posibilidad de que cambien los valores en ejecuciones posteriores)*

A partir de estos atributos, se observa que:

- En este caso, el atributo más importante es **únicamente el código de diagnóstico del cancer de mama** - el diagnóstico de metástasis **pierde peso**.
- Tienen un mayor peso los **atributos numéricos**:
    - **Edad** (`patient_age`) y **IMC** (`bmi`) del paciente.
    - **Mediana del tiempo de transporte de los residentes al trabajo** (`commute_time`).
    - **Porcentaje de la población con edad superior a 40 años** (`age_40s`).
    - **Porcentaje de la población con estudios en STEM** (`education_stem_degree`).
    - **Porcentaje de la población identificada con razas nativas** (`race_native`).
- Los **atributos categóricos identificados** - **raza del paciente** (`patient_race`) y **tipo de seguro médico** (`payer_type`) pierden importancia.
- **El estado de residencia del paciente** (`patient_state`) deja de estar entre los **diez atributos más relevantes**.

El subconjunto de atributos obtenidos por este proceso es el siguiente:

In [191]:
categorical_wrapper = ["breast_cancer_diagnosis_code", "metastatic_cancer_diagnosis_code", "patient_race", "payer_type"]
numerical_wrapper = ["patient_age", "bmi", "commute_time", "race_native", "education_stem_degree", "age_40s"]

### 3.2.3. Conjunto de datos completo

Finalmente, otra opción posible es **utilizar todos los atributos del conjunto de datos** sin ningún tipo de criba o pre-selección. Esta opción tiene ciertas **ventajas** y **desventajas**:
- Por un lado, **algunos modelos funcionan mejor sin selección de variables** - especialmente aquellos que realizan una selección interna (como la **regresión lineal de tipo Lasso** o los **ensembles tipo Random Forest**).
- Por otro lado, **la complejidad excesiva del conjunto de datos** puede afectar al entrenamiento - provocando sobreajustes por la falta de instancias en el conjunto de datos, y perjudicando al rendimiento de los modelos sin selección de atributos interna. Además, **el tiempo de entrenamiento aumenta considerablemente** debido a la mayor complejidad de los modelos entrenados.

En este caso, se incluye principalmente como ***baseline*** para el resto de opciones - pudiendo de esta forma estudiar si **la selección de variables ha mejorado el rendimiento de los modelos**.

In [192]:
categorical_full = X_train.select_dtypes(exclude=np.number).columns.to_list()
numerical_full = X_train.select_dtypes(include=np.number).columns.to_list()

---

<a id="section3-3"></a>

## 3.3. Pre-procesamiento de los datos

El último paso antes del entrenamiento de los modelos es el **preprocesamiento del conjunto de datos** - realizar las transformaciones adecuadas sobre el conjunto de datos para **optimizarlo** de cara al entrenamiento y evaluación de los modelos, con el fin de mejorar su rendimiento.

Los pasos que se van a seguir para pre-procesar los datos - dependiendo del **tipo de datos** - son los siguientes:
- **Atributos categóricos**:
    1. **Imputación**: Se reemplazan todos los valores perdidos por un **valor constante** - `UNKNOWN`.
        - Como se estudió durante el análisis exploratorio, en la mayoría de variables categóricas **tiene sentido tratar los valores perdidos como un valor distinto** - al poder representar valores que no se han podido adquirir o que no se han querido compartir.
    2. **Codificación**: Se transforma la representación de la variable utilizando **`One-Hot Encoding`** - el atributo se divide en **tantos atributos como valores tiene la variable**, donde cada uno de estos nuevos atributos representa si la instancia contiene el valor representado (`1`) o no (`0`).
        - Debido a la gran complejidad de los atributos categóricos (teniendo, en general, **40 o más posibles valores**) y a que **los atributos no son exhaustivos** y pueden haber valores no vistos antes en el conjunto de datos, es necesario **agrupar los valores menos frecuentes** para reducir la dimensionalidad. El **umbral de agrupamiento** será uno de los hiperparámetros a ajustar posteriormente.
- **Atributos numéricos**:
    1. **Imputación**: Se reemplazan todos los valores perdidos por **la mediana del atributo**.
        - En general, el conjunto de datos tiene un gran número de **valores extremos y sesgos**. Utilizar la mediana frente a la media ayuda a crear valores más robustos.
    2. **Escalado**: Se *centran* los valores de los atributos alrededor de la **mediana** y la **desviación estándar**.
        - Normalmente se escala utilizando el **valor promedio**. Ahora bien, para tener un escalado más robusto se utiliza el **rango intercuartil** para el escalado a través de la clase `RobustScaler`.

El problema ahora es que los pasos de preprocesamiento deben ser **idénticos y replicables** en todos los aspectos - tanto entre los experimentos con **distintos modelos y subconjuntos de atributos**, como entre **el entrenamiento y la evaluación** con distintos conjuntos de datos.

Para automatizar este proceso, se definen **`Pipelines`** - cadenas de transformaciones que se aplican **automáticamente** antes de usar los modelos de aprendizaje automático, tanto para **entrenarlos** como para **predecir la variable objetivo para un conjunto de datos**. Es necesario definir `Pipelines` para cada uno de los subconjuntos de atributos que se van a utilizar:

#### - Subconjunto manual

Al tener únicamente atributos categóricos, el `Pipeline` solo contiene las transformaciones asociadas a las variables categóricas. 

Ahora bien, para facilitar la automatización, se **utilizará la estructura dividida por tipos de dato** utilizada por el resto de *Pipelines* - mediante `ColumnTransformer`:

In [194]:
# MANUAL PIPELINE - Categorical values
# Reuses the ColumnTransformer structure to automatize accessing the OneHotEncoder hyperparameters - as min_frequency will be adjusted
# Otherwise, the code would have to take into account the subset of attributes being used to know the inner pipeline structure

preprocessing_manual = ColumnTransformer([
    ("cat", Pipeline([
        ("imputer", SimpleImputer(strategy="constant", fill_value="UNKNOWN")),
        ("onehot", OneHotEncoder(handle_unknown="infrequent_if_exist", min_frequency=100))
    ]), categorical_manual)
])
display(preprocessing_manual)

ColumnTransformer(transformers=[('cat',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(fill_value='UNKNOWN',
                                                                strategy='constant')),
                                                 ('onehot',
                                                  OneHotEncoder(handle_unknown='infrequent_if_exist',
                                                                min_frequency=100))]),
                                 ['breast_cancer_diagnosis_code',
                                  'metastatic_cancer_diagnosis_code',
                                  'patient_race', 'payer_type',
                                  'patient_state'])])

### - Subconjuntos automáticos (*filter* y *wrapper*) y conjunto de datos completo

En estos casos, al tener atributos **categóricos** y **numéricos** el *Pipeline* necesita poder transformar por separado los dos tipos de atributos - utilizando `ColumnTransformer`:

In [195]:
# AUTOMATIC PIPELINE - Filter values
preprocessing_filter = ColumnTransformer([
    ("cat", Pipeline([
        ("imputer", SimpleImputer(strategy="constant", fill_value="UNKNOWN")),
        ("onehot", OneHotEncoder(handle_unknown="infrequent_if_exist", min_frequency=100))
    ]), categorical_filter),
    ("num", Pipeline([
        ("imputer", SimpleImputer(strategy="median")),
        ("scaler", RobustScaler())
    ]), numerical_filter)
])
display(preprocessing_filter)

ColumnTransformer(transformers=[('cat',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(fill_value='UNKNOWN',
                                                                strategy='constant')),
                                                 ('onehot',
                                                  OneHotEncoder(handle_unknown='infrequent_if_exist',
                                                                min_frequency=100))]),
                                 ['breast_cancer_diagnosis_code',
                                  'metastatic_cancer_diagnosis_code',
                                  'payer_type', 'patient_state',
                                  'patient_race']),
                                ('num',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(strategy='median')),
                                                 ('scaler', RobustScaler())]),
                                 ['patient_age', 'labor_force_participation',
                                  'education_bachelors', 'family_dual_income',
                                  'education_college_or_above'])])

In [196]:
# AUTOMATIC PIPELINE - Wrapper
preprocessing_wrapper = ColumnTransformer([
    ("cat", Pipeline([
        ("imputer", SimpleImputer(strategy="constant", fill_value="UNKNOWN")),
        ("onehot", OneHotEncoder(handle_unknown="infrequent_if_exist", min_frequency=100))
    ]), categorical_wrapper),
    ("num", Pipeline([
        ("imputer", SimpleImputer(strategy="median")),
        ("scaler", RobustScaler())
    ]), numerical_wrapper)
])
display(preprocessing_wrapper)

ColumnTransformer(transformers=[('cat',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(fill_value='UNKNOWN',
                                                                strategy='constant')),
                                                 ('onehot',
                                                  OneHotEncoder(handle_unknown='infrequent_if_exist',
                                                                min_frequency=100))]),
                                 ['breast_cancer_diagnosis_code',
                                  'metastatic_cancer_diagnosis_code',
                                  'patient_race', 'payer_type']),
                                ('num',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(strategy='median')),
                                                 ('scaler', RobustScaler())]),
                                 ['patient_age', 'bmi', 'commute_time',
                                  'race_native', 'education_stem_degree',
                                  'age_40s'])])

In [197]:
# FULL PIPELINE
# AUTOMATIC PIPELINE - Wrapper
preprocessing_full = ColumnTransformer([
    ("cat", Pipeline([
        ("imputer", SimpleImputer(strategy="constant", fill_value="UNKNOWN")),
        ("onehot", OneHotEncoder(handle_unknown="infrequent_if_exist", min_frequency=100))
    ]), categorical_full),
    ("num", Pipeline([
        ("imputer", SimpleImputer(strategy="median")),
        ("scaler", RobustScaler())
    ]), numerical_full)
])
display(preprocessing_full)

ColumnTransformer(transformers=[('cat',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(fill_value='UNKNOWN',
                                                                strategy='constant')),
                                                 ('onehot',
                                                  OneHotEncoder(handle_unknown='infrequent_if_exist',
                                                                min_frequency=100))]),
                                 ['patient_race', 'payer_type', 'patient_state',
                                  'patient_zip3', 'Region', 'Division',
                                  'patient_gender',
                                  'breast_cancer_diagnosis_code',
                                  'breast_cancer_d...
                                  'age_60s', 'age_70s', 'age_over_80', 'male',
                                  'female', 'married', 'divorced',
                                  'never_married', 'widowed', 'family_size',
                                  'family_dual_income',
                                  'income_household_median',
                                  'income_household_under_5',
                                  'income_household_5_to_10',
                                  'income_household_10_to_15',
                                  'income_household_15_to_20',
                                  'income_household_20_to_25',
                                  'income_household_25_to_35',
                                  'income_household_35_to_50', ...])])

---

<a id="section4"></a>

# 4. Selección de modelos de regresión e hiperparámetros

<a id="section5"></a>

# 5. Experimentación

<a id="section6"></a>

# 6. Análisis de resultados